In [2]:
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"


pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)

data = pd.read_csv('DataSetForPhishingVSBenignUrl.csv', header=0)

data.head(10)

,Querylength,domain_token_count,path_token_count,avgdomaintokenlen,longdomaintokenlen,avgpathtokenlen,tld,charcompvowels,charcompace,ldl_url,ldl_domain,ldl_path,ldl_filename,ldl_getArg,dld_url,dld_domain,dld_path,dld_filename,dld_getArg,urlLen,domainlength,pathLength,subDirLen,fileNameLen,this.fileExtLen,ArgLen,pathurlRatio,ArgUrlRatio,argDomanRatio,domainUrlRatio,pathDomainRatio,argPathRatio,executable,isPortEighty,NumberofDotsinURL,ISIpAddressInDomainName,CharacterContinuityRate,LongestVariableValue,URL_DigitCount,host_DigitCount,Directory_DigitCount,File_name_DigitCount,Extension_DigitCount,Query_DigitCount,URL_Letter_Count,host_letter_count,Directory_LetterCount,Filename_LetterCount,Extension_LetterCount,Query_LetterCount,LongestPathTokenLength,Domain_LongestWordLength,Path_LongestWordLength,sub-Directory_LongestWordLength,Arguments_LongestWordLength,URL_sensitiveWord,URLQueries_variable,spcharUrl,delimeter_Domain,delimeter_path,delimeter_Count,NumberRate_URL,NumberRate_Domain,NumberRate_DirectoryName,NumberRate_FileName,NumberRate_Extension,NumberRate_AfterPath,SymbolCount_URL,SymbolCount_Domain,SymbolCount_Directoryname,SymbolCount_FileName,SymbolCount_Extension,SymbolCount_Afterpath,Entropy_URL,Entropy_Domain,Entropy_DirectoryName,Entropy_Filename,Entropy_Extension,Entropy_Afterpath,URL_Type_obf_Type
0,0,4,5,5.5,14,4.400000,4,8,3,0,0,0,0,0,0,0,0,0,0,58,25,26,26,13,1,2,0.448276,0.034483,0.08,0.431034,1.04,0.0769231,0,-1,5,-1,0.6,-1,1,0,0,0,1,-1,47,22,8,13,0,-1,13,14,13,5,-1,0,0,3,0,2,-1,0.017241,0.0,0.0,0.066667,1.0,-1.0,8,3,2,1,0,-1,0.726298,0.784493,0.894886,0.850608,NaN,-1.0,Defacement
1,0,4,5,5.5,14,6.000000,4,12,4,0,0,0,0,0,0,0,0,0,0,66,25,34,34,2,2,2,0.515151,0.030303,0.08,0.378788,1.36,0.0588235,0,-1,4,-1,0.6,-1,0,0,0,0,0,-1,56,22,8,13,9,-1,13,14,13,5,-1,0,0,4,0,1,-1,0.000000,0.0,0.0,0.000000,NaN,-1.0,8,3,3,0,0,-1,0.688635,0.784493,0.814725,0.859793,0.0,-1.0,Defacement
2,0,4,5,5.5,14,5.800000,4,12,5,0,0,0,0,0,0,0,0,0,0,65,25,33,33,2,2,2,0.507692,0.030769,0.08,0.384615,1.32,0.0606061,0,-1,4,-1,0.6,-1,0,0,0,0,0,-1,55,22,8,13,8,-1,13,14,13,5,-1,0,0,4,0,1,-1,0.000000,0.0,0.0,0.000000,NaN,-1.0,8,3,3,0,0,-1,0.695049,0.784493,0.814725,0.801880,0.0,-1.0,Defacement
3,0,4,12,5.5,14,5.500000,4,32,16,0,0,0,0,0,0,0,0,0,0,109,25,77,77,2,2,2,0.706422,0.018349,0.08,0.229358,3.08,0.025974,0,-1,4,-1,0.6,-1,0,0,0,0,0,-1,92,22,8,13,45,-1,52,14,13,13,-1,0,0,4,0,8,-1,0.000000,0.0,0.0,0.000000,NaN,-1.0,8,3,3,0,0,-1,0.640130,0.784493,0.814725,0.663210,0.0,-1.0,Defacement
4,0,4,6,5.5,14,7.333334,4,18,11,0,0,0,0,0,0,0,0,0,0,81,25,49,49,2,2,2,0.604938,0.024691,0.08,0.308642,1.96,0.0408163,0,-1,4,-1,0.6,-1,0,0,0,0,0,-1,70,22,8,13,23,-1,24,14,13,13,-1,0,0,4,0,2,-1,0.000000,0.0,0.0,0.000000,NaN,-1.0,8,3,3,0,0,-1,0.681307,0.784493,0.814725,0.804526,0.0,-1.0,Defacement
5,0,4,8,5.5,14,6.500000,4,22,10,0,0,0,0,0,0,0,0,0,0,91,25,59,59,2,2,2,0.648352,0.021978,0.08,0.274725,2.36,0.0338983,0,-1,4,-1,0.6,-1,0,0,0,0,0,-1,78,22,8,13,31,-1,34,14,15,13,-1,0,0,4,0,4,-1,0.000000,0.0,0.0,0.000000,NaN,-1.0,8,3,3,0,0,-1,0.666676,0.784493,0.814725,0.755658,0.0,-1.0,Defacement
6,0,4,5,5.5,14,7.800000,4,17,10,0,0,0,0,0,0,0,0,0,0,75,25,43,43,2,2,2,0.573333,0.026667,0.08,0.333333,1.72,0.0465116,0,-1,4,-1,0.6,-1,0,0,0,0,0,-1,65,22,8,13,18,-1,18,14,18,5,-1,0,0,4,0,1,-1,0.000000,0.0,0.0,0.000000,NaN,-1.0,8,3,3,0,0,-1,0.682440,0.784493,0.814725,0.766719,0.0,-1.0,Defacement
7,0,4,7,5.5,14,6.285714,4,16,9,0,0,0,0,0,0,0,0,0,0,82,25,50,50,2,2,2,0.609756,0.024390,0.08,0.304878,2.00,0.04,0,-1,4,-1,0.6,-1,8,0,0,0,8,-1,62,22,8,13,15,-1,25,14,13,13,-1,0,0,4,0,3,-1,0.097561,0.0,0.0,0.320000,NaN,-1.0,8,3,3,0,0,-1,0.709396,0.784493,0.814725,0.797498,0.0,-1.0,Defacement
8,0,4,6,5.5,14,6.500000,4,16,10,0,0,0,0,0,0,0,0,0,0,76,25,44,44,2,2,2,0.578947,0.026316,0.08,0.328947,1.76,0.0454545,0,-1,4,-1,0.6,-1,0,0,0,0,0,-1,65,22,8,13,18,-1,19,14,13,13,-1,0,0,4,0,2,-1,0.000000,0.0,0.0,0.000000,NaN,-1.0,8,3,3,0,0,-1,0.678242,0.784493,0.814725,0.732258,0.0,-1.0,Defacement
9,0,4,5,5.5,14,3.60000

### Handle columns with Nulls in the current data set

In [3]:
data.isnull().sum()

Querylength                            0
domain_token_count                     0
path_token_count                       0
avgdomaintokenlen                      0
longdomaintokenlen                     0
avgpathtokenlen                      280
tld                                    0
charcompvowels                         0
charcompace                            0
ldl_url                                0
ldl_domain                             0
ldl_path                               0
ldl_filename                           0
ldl_getArg                             0
dld_url                                0
dld_domain                             0
dld_path                               0
dld_filename                           0
dld_getArg                             0
urlLen                                 0
domainlength                           0
pathLength                             0
subDirLen                              0
fileNameLen                            0
this.fileExtLen 

In [4]:
data_clean_na = data.dropna()         #(subset=['Entropy_DirectoryName'])

In [16]:
lst = ['phishing' , 'benign']

data_clean = data_clean_na[data_clean_na.URL_Type_obf_Type.isin(lst)  ]
data_clean['URL_Type_obf_Type'].unique()


array(['benign', 'phishing'], dtype=object)

In [17]:
data_clean.dtypes

Querylength                          int64
domain_token_count                   int64
path_token_count                     int64
avgdomaintokenlen                  float64
longdomaintokenlen                   int64
avgpathtokenlen                    float64
tld                                  int64
charcompvowels                       int64
charcompace                          int64
ldl_url                              int64
ldl_domain                           int64
ldl_path                             int64
ldl_filename                         int64
ldl_getArg                           int64
dld_url                              int64
dld_domain                           int64
dld_path                             int64
dld_filename                         int64
dld_getArg                           int64
urlLen                               int64
domainlength                         int64
pathLength                           int64
subDirLen                            int64
fileNameLen

In [18]:
# split into input and output elements


data_numpy = data_clean.values
X, y = data_numpy[:, :-1], data_numpy[:, -1]


#X = data_clean.drop('URL_Type_obf_Type',axis = 'columns')
#y =  data_clean.URL_Type_obf_Type

## SMOT Oversampling the data to removed skewed classes

In [19]:
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from collections import Counter


y = LabelEncoder().fit_transform(y)

# transform the dataset
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)
# summarize distribution

In [20]:
from matplotlib import pyplot

counter = Counter(y)
for k,v in counter.items():
    per = v / len(y) * 100
    print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
# plot the distribution
pyplot.bar(counter.keys(), counter.values())
pyplot.show()

Class=0, n=4014 (50.000%)
Class=1, n=4014 (50.000%)


<BarContainer object of 2 artists>

<Figure size 640x480 with 1 Axes>

### Test and Training Split and Scaling the data

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1, stratify=y)


sc = StandardScaler()
sc.fit(X_train)

X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

print('Labels count in y:', np.bincount(y))
print('Labels count in y_train:', np.bincount(y_train))

print('Labels count in y_test:', np.bincount(y_test))

StandardScaler()

Labels count in y: [4014 4014]
Labels count in y_train: [3211 3211]
Labels count in y_test: [803 803]


### TRAIN MODEL & CREATE PREDICTIONS USING SKLEARN - GINI CRITERION 

In [73]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import metrics


#developing a model with gini , tree and adboost


clf_gini_d1 = DecisionTreeClassifier(criterion = 'gini' , max_depth = 1 , splitter = 'best')
clf_gini_d3 = DecisionTreeClassifier(criterion = 'gini' , max_depth = 3 , splitter = 'best')
clf_gini_d6 = DecisionTreeClassifier(criterion = 'gini' , max_depth = 6 , splitter = 'best')
clf_gini_d9 = DecisionTreeClassifier(criterion = 'gini' , max_depth = 9 , splitter = 'best')
clf_gini_d12 = DecisionTreeClassifier(criterion = 'gini' , max_depth = 12 , splitter = 'best')
clf_gini_d15 = DecisionTreeClassifier(criterion = 'gini' , max_depth = 15 , splitter = 'best')
clf_gini_d18 = DecisionTreeClassifier(criterion = 'gini' , max_depth = 18 , splitter = 'best')



clf_g_d1 = AdaBoostClassifier(base_estimator = clf_gini_d1 ,n_estimators=50, random_state=0)
clf_g_d3 = AdaBoostClassifier(base_estimator = clf_gini_d3 ,n_estimators=50, random_state=0)
clf_g_d6 = AdaBoostClassifier(base_estimator = clf_gini_d6 ,n_estimators=50, random_state=0)
clf_g_d9 = AdaBoostClassifier(base_estimator = clf_gini_d9 ,n_estimators=50, random_state=0)
clf_g_d12 = AdaBoostClassifier(base_estimator = clf_gini_d12 ,n_estimators=50, random_state=0)
clf_g_d15 = AdaBoostClassifier(base_estimator = clf_gini_d15 ,n_estimators=50, random_state=0)
clf_g_d18 = AdaBoostClassifier(base_estimator = clf_gini_d18 ,n_estimators=50, random_state=0)



clf_ada_g_1 = clf_g_d1.fit(X_train_std,y_train)
clf_ada_g_3 = clf_g_d3.fit(X_train_std,y_train)
clf_ada_g_6 = clf_g_d6.fit(X_train_std,y_train)
clf_ada_g_9 = clf_g_d9.fit(X_train_std,y_train)
clf_ada_g_12 = clf_g_d12.fit(X_train_std,y_train)
clf_ada_g_15 = clf_g_d15.fit(X_train_std,y_train)
clf_ada_g_18 = clf_g_d18.fit(X_train_std,y_train)


y_pred_g_1 = clf_ada_g_1.predict(X_test_std)
y_pred_g_3 = clf_ada_g_3.predict(X_test_std)
y_pred_g_6 = clf_ada_g_6.predict(X_test_std)
y_pred_g_9 = clf_ada_g_9.predict(X_test_std)
y_pred_g_12 = clf_ada_g_12.predict(X_test_std)
y_pred_g_15 = clf_ada_g_15.predict(X_test_std)
y_pred_g_18 = clf_ada_g_18.predict(X_test_std)




In [74]:
print("Accuracy Gini 1:",metrics.accuracy_score(y_test, y_pred_g_1))
print("Accuracy Gini 3:",metrics.accuracy_score(y_test, y_pred_g_3))
print("Accuracy Gini 6:",metrics.accuracy_score(y_test, y_pred_g_6))
print("Accuracy Gini 9:",metrics.accuracy_score(y_test, y_pred_g_9))
print("Accuracy Gini 12:",metrics.accuracy_score(y_test, y_pred_g_12))
print("Accuracy Gini 15:",metrics.accuracy_score(y_test, y_pred_g_15))
print("Accuracy Gini 18:",metrics.accuracy_score(y_test, y_pred_g_18))

Accuracy Gini 1: 0.9645080946450809
Accuracy Gini 3: 0.975093399750934
Accuracy Gini 6: 0.9782067247820673
Accuracy Gini 9: 0.9788293897882939
Accuracy Gini 12: 0.9825653798256538
Accuracy Gini 15: 0.9813200498132005
Accuracy Gini 18: 0.9838107098381071


### TRAIN MODEL & CREATE PREDICTIONS USING SKLEARN - ENTROPY

In [76]:
clf_entropy_d1 = DecisionTreeClassifier(criterion = 'entropy' , max_depth = 1 , splitter = 'best')
clf_entropy_d3 = DecisionTreeClassifier(criterion = 'entropy' , max_depth = 3 , splitter = 'best')
clf_entropy_d6 = DecisionTreeClassifier(criterion = 'entropy' , max_depth = 6 , splitter = 'best')
clf_entropy_d9 = DecisionTreeClassifier(criterion = 'entropy' , max_depth = 9 , splitter = 'best')
clf_entropy_d12 = DecisionTreeClassifier(criterion = 'entropy' , max_depth = 12 , splitter = 'best')
clf_entropy_d15 = DecisionTreeClassifier(criterion = 'entropy' , max_depth = 15 , splitter = 'best')
clf_entropy_d18 = DecisionTreeClassifier(criterion = 'entropy' , max_depth = 18 , splitter = 'best')



clf_e_d1 = AdaBoostClassifier(base_estimator = clf_entropy_d1 ,n_estimators=50, random_state=0)
clf_e_d3 = AdaBoostClassifier(base_estimator = clf_entropy_d3 ,n_estimators=50, random_state=0)
clf_e_d6 = AdaBoostClassifier(base_estimator = clf_entropy_d6 ,n_estimators=50, random_state=0)
clf_e_d9 = AdaBoostClassifier(base_estimator = clf_entropy_d9 ,n_estimators=50, random_state=0)
clf_e_d12 = AdaBoostClassifier(base_estimator = clf_entropy_d12 ,n_estimators=50, random_state=0)
clf_e_d15 = AdaBoostClassifier(base_estimator = clf_entropy_d15 ,n_estimators=50, random_state=0)
clf_e_d18 = AdaBoostClassifier(base_estimator = clf_entropy_d18 ,n_estimators=50, random_state=0)



clf_ada_e_1 = clf_e_d1.fit(X_train_std,y_train)
clf_ada_e_3 = clf_e_d3.fit(X_train_std,y_train)
clf_ada_e_6 = clf_e_d6.fit(X_train_std,y_train)
clf_ada_e_9 = clf_e_d9.fit(X_train_std,y_train)
clf_ada_e_12 = clf_e_d12.fit(X_train_std,y_train)
clf_ada_e_15 = clf_e_d15.fit(X_train_std,y_train)
clf_ada_e_18 = clf_e_d18.fit(X_train_std,y_train)



y_pred_e_1 = clf_ada_e_1.predict(X_test_std)
y_pred_e_3 = clf_ada_e_3.predict(X_test_std)
y_pred_e_6 = clf_ada_e_6.predict(X_test_std)
y_pred_e_9 = clf_ada_e_9.predict(X_test_std)
y_pred_e_12 = clf_ada_e_12.predict(X_test_std)
y_pred_e_15 = clf_ada_e_15.predict(X_test_std)
y_pred_e_18 = clf_ada_e_18.predict(X_test_std)


In [77]:
print("Accuracy Entropy 1:",metrics.accuracy_score(y_test, y_pred_e_1))
print("Accuracy Entropy 3:",metrics.accuracy_score(y_test, y_pred_e_3))
print("Accuracy Entropy 6:",metrics.accuracy_score(y_test, y_pred_e_6))
print("Accuracy Entropy 9:",metrics.accuracy_score(y_test, y_pred_e_9))
print("Accuracy Entropy 12:",metrics.accuracy_score(y_test, y_pred_e_12))
print("Accuracy Entropy 15:",metrics.accuracy_score(y_test, y_pred_e_15))
print("Accuracy Entropy 18:",metrics.accuracy_score(y_test, y_pred_e_18))

Accuracy Entropy 1: 0.9676214196762142
Accuracy Entropy 3: 0.9800747198007472
Accuracy Entropy 6: 0.9794520547945206
Accuracy Entropy 9: 0.9825653798256538
Accuracy Entropy 12: 0.9819427148194272
Accuracy Entropy 15: 0.9694894146948941
Accuracy Entropy 18: 0.9694894146948941


#### ANSWER : The accuracy using Ensemble is much higer at a greater performance as compared to using Decision Tree independently. The max accuracy reached with Ensemble was around 98% however around 88% with only using decision tree without ensemble.